<a href="https://colab.research.google.com/github/Crypto2000Bytes/Pyomo-models/blob/main/Generators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q pyomo


In [10]:
!apt-get install -y -qq glpk-utils


In [19]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
import numpy as np
import time

#importeer de excel.files

dataGen = pd.read_excel("/content/drive/MyDrive/inputs.xlsx", sheet_name = "gen")
print(dataGen)
dataLoad = pd.read_excel("/content/drive/MyDrive/inputs.xlsx", sheet_name = "load")
print(dataLoad)
Ng = len(dataGen)
#model
model = pyo.ConcreteModel()

#Variabele, er is een variabele namelij Pg: energie van een generator
model.Pg = pyo.Var(range(Ng), bounds = (0, None))
Pg = model.Pg

#Restricties:

# 1.Balans
Pg_sum = sum([Pg[i] for i in dataGen.id])
Pd = dataLoad.value
Pd_sum = sum([Pd[i] for i in dataLoad.id])
model.balans = pyo.Constraint(expr = Pg_sum == Pd_sum)

#2. Laadpunt 0 wordt van energie voorzien door generator 0 & 3
model.con1 = pyo.Constraint(expr = Pg[0] + Pg[3] >= Pd[0])

# 3. limieten van de generatoren
model.limits = pyo.ConstraintList()
for i in dataGen.id:
  model.limits.add(expr = Pg[i] <= dataGen.limit[i])

# Objective
model.obj = pyo.Objective(expr = sum([dataGen.cost[i]*Pg[i] for i in dataGen.id]))

# Solver 
tijd_start = time.time()
opt = pyo.SolverFactory('glpk')
opt.solve(model)
eind_tijd = time.time()-tijd_start

model.pprint()
dataGen['Pg'] = [pyo.value(Pg[i]) for i in dataGen.id]
print("tijd", eind_tijd)
print(dataGen)

   id  limit  cost
0   0     20  0.10
1   1     10  0.05
2   2     40  0.30
3   3     50  0.40
4   4      5  0.01
   id  value
0   0     50
1   1     20
2   2     30
2 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    limits_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}

1 Var Declarations
    Pg : Size=5, Index=Pg_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  20.0 :  None : False : False :  Reals
          1 :     0 :  10.0 :  None : False : False :  Reals
          2 :     0 :  35.0 :  None : False : False :  Reals
          3 :     0 :  30.0 :  None : False : False :  Reals
          4 :     0 :   5.0 :  None : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Acti